In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import cv2
import scipy.io
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import os

In [ ]:
def l2_samples(samples, samples_gt):
    val = 0
    for i in range(4):
        for j in range(4):
            val += np.square(samples[i][j] - samples_gt[i][j]).mean()

    return val / 16

def ssim_samples(samples, samples_gt):
    val = 0
    for i in range(4):
        for j in range(4):
            val += ssim(samples[i][j], samples_gt[i][j], channel_axis=2)

    return val / 16

def psnr_samples(samples, samples_gt):
    val = 0
    for i in range(4):
        for j in range(4):
            val += psnr(samples[i][j], samples_gt[i][j])

    return val / 16

In [ ]:
def fft_denoising_single(image, mask):
    image = image.transpose((2, 0, 1))
    # Apply Fourier Transform to each channel
    f_transform_channels = [np.fft.fft2(channel) for channel in image]
    f_transform_shifted_channels = [np.fft.fftshift(f_transform) for f_transform in f_transform_channels]
    magnitude_spectra = [20 * np.log(np.abs(f_transform_shifted) + 1e-10) for f_transform_shifted in f_transform_shifted_channels]

    # Apply mask 
    f_transform_shifted_channels = [f_transform_shifted * mask for f_transform_shifted in f_transform_shifted_channels]
    
    # Inverse DFT to each channel
    f_ishift_channels = [np.fft.ifftshift(f_transform_shifted) for f_transform_shifted in f_transform_shifted_channels]
    img_back_channels = [np.fft.ifft2(f_ishift) for f_ishift in f_ishift_channels]
    img_back_channels = [np.abs(img_back) for img_back in img_back_channels]

    # Merge the channels back into a color image
    img_back = cv2.merge(img_back_channels)

    return img_back.astype(np.uint8)

def fft_samples(samples, mask):
    out = []
    for i in range(4):
        temp = []
        for j in range(4):
            temp.append(fft_denoising_single(samples[i, j], mask))
        out.append(temp)

    return out


In [ ]:
def mask_ellipse(a, b):
    mask = np.zeros((256, 256))
    x, y = np.ogrid[:256, :256]
    mask_area = ((x - 127) / a) ** 2 + ((y - 127) / b) ** 2 <= 1
    mask[mask_area] = 1

    return mask

def mask_diamond(a, b):
    mask = np.zeros((256, 256))
    x, y = np.ogrid[:256, :256]
    mask_area = abs((x - 127) / a) + abs((y - 127) / b) <= 1
    mask[mask_area] = 1

    return mask

def mask_star(a, b):
    mask = np.zeros((256, 256))
    x, y = np.ogrid[:256, :256]
    mask_area = abs((x - 127) / a) ** 0.5 + abs((y - 127) / b) ** 0.5 <= 1
    mask[mask_area] = 1

    return mask


In [ ]:
dataset = 'SIDD'
method = 'FFT'
mask_type = 'Star'

In [ ]:
# get current directory
path = os.getcwd()
 
# prints parent directory
par_path = os.path.abspath(os.path.join(path, os.pardir))

# .mat files
if dataset == 'SIDD':
    val_noisy = par_path + '/data/sidd/validation/val_noisy.mat'  # Noisy
    val_gt = par_path + '/data/sidd/validation/val_gt.mat'  # GT
    tag_noisy = 'ValidationNoisyBlocksSrgb'
    tag_gt = 'ValidationGtBlocksSrgb'
elif dataset == 'DIV2K_GSN_10':
    val_noisy = par_path + '/data/div2k_medium_gsn_10/validation/val_noisy.mat'  # Noisy
    val_gt = par_path + '/data/div2k_medium_gsn_10/validation/val_gt.mat'  # GT
    tag_noisy = 'val_ng'
    tag_gt = 'val_gt'
elif dataset == 'DIV2K_SNP_10':
    val_noisy = par_path + '/data/div2k_medium_snp_10/validation/val_noisy.mat'  # Noisy
    val_gt = par_path + '/data/div2k_medium_snp_10/validation/val_gt.mat'  # GT
    tag_noisy = 'val_ng'
    tag_gt = 'val_gt'

# Load .mat files
val_noisy_mat = np.array(scipy.io.loadmat(val_noisy)[tag_noisy])
val_gt_mat = np.array(scipy.io.loadmat(val_gt)[tag_gt])

In [ ]:
# Sample Images
image_samples = [1, 10, 17, 23]
image_crops = [2, 4, 7, 11]
samples_noisy = np.array([[val_noisy_mat[i, j] for j in image_crops] for i in image_samples])
samples_gt = np.array([[val_gt_mat[i, j] for j in image_crops] for i in image_samples])

In [ ]:
A = list(range(1, 50, 1))
B = list(range(1, 50, 1))

In [ ]:
print('ORIGINAL VALUES\n')

print('L2: ', l2_samples(samples_noisy, samples_gt))
print('SSIM: ', ssim_samples(samples_noisy, samples_gt))
print('PSNR: ', psnr_samples(samples_noisy, samples_gt))

In [ ]:
best_loss = 1e4
best_a = None
best_b = None
best_index = None

metrics = np.zeros((len(A), len(B)))
for i, a in enumerate(A):
    for j, b in enumerate(B):
        if mask_type == 'Ellipse':
            mask = mask_ellipse(a, b)
        elif mask_type == 'Diamond':
            mask = mask_diamond(a, b)
        else:
            mask = mask_star(a, b)

        test = fft_samples(samples_noisy, mask)

        avg_loss = 1 - ssim_samples(test, samples_gt)
        metrics[i, j] = avg_loss

        if avg_loss < best_loss:
            best_index = [i, j]
            best_a = a
            best_b = b
            best_loss = avg_loss

print(best_a, best_b)

In [ ]:
mask_type

In [ ]:
metrics_normalized = (metrics - metrics.min()) / (metrics.max() - metrics.min())

fig = go.Figure(data=go.Heatmap(z=metrics_normalized.T, x=A, y=B, colorscale='viridis'))
# Add a star annotation for the minimum value
fig.add_annotation(
    x=best_a, y=best_b,
    text="★ min",
    showarrow=False,
    font=dict(size=10, color="red")
)    
fig.update_layout(height=800, width=800, 
    xaxis_title='A',
    yaxis_title='B'
)


In [ ]:
if mask_type == 'Ellipse':
    mask = mask_ellipse(best_a, best_b)
elif mask_type == 'Diamond':
    mask = mask_diamond(best_a, best_b)
else:
    mask = mask_star(best_a, best_b)

denoised = fft_samples(samples_noisy, mask_ellipse(best_a, best_b))

print('Denoised VALUES\n')

print('L2: ', l2_samples(denoised, samples_gt))
print('SSIM: ', ssim_samples(denoised, samples_gt))
print('PSNR: ', psnr_samples(denoised, samples_gt))

In [ ]:
# Create subplot figure with 2 rows and 2 columns
fig = make_subplots(rows=4, cols=6, subplot_titles=(
    "Noisy 01", "Denoised 01", "GT 01", "Noisy 02", "Denoised 02", "GT 02", 
    "Noisy 03", "Denoised 03", "GT 03", "Noisy 04", "Denoised 04", "GT 04",
    "Noisy 05", "Denoised 05", "GT 05", "Noisy 06", "Denoised 06", "GT 06", 
    "Noisy 07", "Denoised 07", "GT 07", "Noisy 08", "Denoised 08", "GT 08"
))

# Add images to the subplots
fig.add_trace(go.Image(z=samples_noisy[0][1]), row=1, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[0][1]), row=1, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[0][1]), row=1, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][0]), row=1, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[1][0]), row=1, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[1][0]), row=1, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[1][3]), row=2, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[1][3]), row=2, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[1][3]), row=2, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[2][0]), row=2, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[2][0]), row=2, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[2][0]), row=2, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][0]), row=3, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][0]), row=3, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][0]), row=3, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][1]), row=3, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][1]), row=3, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][1]), row=3, col=6)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][2]), row=4, col=1)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][2]), row=4, col=2)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][2]), row=4, col=3)  # Ground Truth

fig.add_trace(go.Image(z=samples_noisy[3][3]), row=4, col=4)  # Noisy Image
fig.add_trace(go.Image(z=denoised[3][3]), row=4, col=5)  # Denoised Image
fig.add_trace(go.Image(z=samples_gt[3][3]), row=4, col=6)  # Ground Truth

# Remove axes for all subplots
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.update_layout(height=1000, 
                  width=1200, 
                  # title_text="Noisy and Ground Truth Images",
                  margin=dict(l=40, r=40, t=40, b=20),  # Reduce margins
)
fig.show()

In [ ]:
sample_noisy = samples_noisy[0, 3]
sample_gt = samples_gt[0, 3]
sample_denoised = denoised[0][3]

In [ ]:
from PIL import Image

# Convert the NumPy array to a PIL Image object
img = Image.fromarray(sample_denoised)

# Save the image
img.save("den_star.png")